# Rosenkranz-Schlüssel: Interaktive Q-Simulation für Verkehrssicherheit

Diese Simulation optimiert Risikofaktoren (z.B. Dichte, Geschwindigkeit) mit Q-Learning-Approximation. Passe Parameter mit Sliders an und unlocke 'Schlüssel' für sichere Routen.

**Basierend auf früherem Code (z.B. aus Q-Simulation_verkehr.py), fixed und erweitert.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
# Optional: Import aus bestehender Datei (falls relevant)
# %run ./Q-Simulation_verkehr.py  # Uncomment, wenn Funktionen reuse

In [ ]:
# Kernfunktionen (fixed: Definierte Vars, korrigierte Logik)
def skalenvariant_reichweite(risiko):
    return np.sqrt(1 / risiko)  # Skalenvariante Reichweite (z.B. Sicht in Nebel)

def uberlebens_konstante(Q, risiko_factor, K=1.0):
    return K * (Q / risiko_factor)  # Konstante für Überlebenswahrscheinlichkeit

def rosenkranz_approximation(Q_start, iterations, unsicherheit=0.1):
    Q_approx = Q_start
    for i in range(1, iterations + 1):
        Q_approx += ((-1)**(i+1) * (unsicherheit**i / i))  # Alternierende Serie für Approx (inspiriert von ln(1+x))
    return Q_approx

In [ ]:
# Interaktive Simulation (Sliders für Verkehrs-Params, Unlock-Logik)
@interact(
    samples=IntSlider(min=50, max=500, step=50, value=100, description='Samples:'),
    min_risiko=FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05, description='Min Risiko (Dichte):'),
    max_risiko=FloatSlider(min=0.3, max=1.0, step=0.05, value=0.5, description='Max Risiko (Dichte):'),
    iterations=IntSlider(min=5, max=50, step=5, value=10, description='Iterationen:'),
    unsicherheit=FloatSlider(min=0.05, max=0.5, step=0.05, value=0.1, description='Unsicherheit:')
)
def simulate_rosenkranz_schluessel(samples, min_risiko, max_risiko, iterations, unsicherheit):
    scale = np.linspace(1, 100, samples)  # Skala: z.B. Geschwindigkeit km/h
    risikofaktor = np.random.uniform(min_risiko, max_risiko, samples)  # Risiko: z.B. Verkehrsdichte
    Q_start = 1.0  # Initial-Q
    Q_werte = []
    for s, risiko in zip(scale, risikofaktor):
        reichweite = skalenvariant_reichweite(risiko)
        Q_basis = uberlebens_konstante(Q_start, risiko)
        Q_optim = rosenkranz_approximation(Q_basis, iterations, unsicherheit)
        Q_werte.append(Q_optim * (s / 100))  # Skaliere für Realismus
    
    plt.figure(figsize=(12, 6))
    plt.plot(scale, Q_werte, 'b-', label='Optimierter Q-Schlüssel')
    plt.xlabel('Geschwindigkeit/Skala (km/h)')
    plt.ylabel('Q-Wert (Sicherheit)')
    plt.title('Rosenkranz-Schlüssel: Verkehrsrisiko-Optimierung')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    mean_Q = np.mean(Q_werte)
    print(f'Durchschnittlicher Q-Schlüssel: {mean_Q:.2f}')
    if mean_Q > 10:
        print('Schlüssel unlocked: Sichere Route/Ampel freigeschaltet!')
    else:
        print('Schlüssel locked: Risiko zu hoch – optimiere!')